# Table of Contents
* [Import](#import)
* [Distribution](#distribution)
* [EDA](#eda)
* [Look at an Example Sequence](#example)
* [Aggregation of Data](#aggregation)
* [Target vs Features](#target_feature)
* [Fit Model](#model)
* [Model Transparency](#model_trans)
* [Apply model on test set and prepare submission](#submit)


In [ ]:
# import packages
suppressWarnings(library(tidyverse))
suppressWarnings(library(readr))
suppressWarnings(library(corrplot))
suppressWarnings(library(h2o))

In [ ]:
# plot options
options(repr.plot.width = 12, repr.plot.height = 6)

<a id='import'></a>
# Import

**Data Fetching**

Data fetching involves extraction of data from train.csv file which comprises of 36 columns. Sequence and State are the output columns. This is the target which you are trying to predict. Sequence, Subject, Sensor and Step are the input columns. 



In [ ]:
# import training data
time1 <- Sys.time()
df_train <- read.csv('../input/tabular-playground-series-apr-2022/train.csv')
time2 <- Sys.time()
print(time2-time1)

# Structure of the train dataset

In [ ]:
#find data types of train
str(df_train)

In [ ]:
# load sequence label for training
df_train_lab <- read.csv('../input/tabular-playground-series-apr-2022/train_labels.csv')


In [ ]:
# import testing data
time1 <- Sys.time()
df_test  <- read.csv('../input/tabular-playground-series-apr-2022/test.csv')
time2 <- Sys.time()
print(time2-time1)

**Description of the data columns:**

***train.csv*** - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants

* sequence - a unique id for each sequence
* subject - a unique id for the subject in the experiment
* step - time step of the recording, in one second intervals
* sensor_00 - sensor_12 - the value for each of the thirteen sensors at that time step


**train_labels.csv**** - the class label for each sequence.

* sequence - the unique id for each sequence.
* state - the state associated to each sequence. This is the target which you are trying to predict.

In [ ]:
#find max values of seuence and subject , so we can get idea that these two columns are in sequence or not
max(df_train$sequence)
max(df_train$subject)
max(df_train_lab$sequence)
min(df_test$sequence)
min(df_test$subject)

So we conclude that,
**Insights 1: Sequence and Subject features are series**

In [ ]:
# import submission template
df_sub <- read.csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')

In [ ]:
# print the shape of datasets.
dim(df_train)
dim(df_test)
dim(df_sub)
dim(df_train_lab)

In [ ]:
# we check here for null values
is.null(df_train)
is.null(df_test)
is.null(df_sub)
is.null(df_train_lab)
# we found there is no null values

In [ ]:
# checked for duplicated values in every dataframes
sum(duplicated(df_train))
sum(duplicated(df_test))
sum(duplicated(df_sub))
sum(duplicated(df_train_lab))
# from the result we can see that there is no dulpicate values

# Distribution

In [ ]:
#distribution plot
d <- density(df_train$sequence) 
plot(d)

In [ ]:
d <- density(df_train$subject) 
plot(d) 

So found that,
**Insights 2: Sequence feature has uniform distribution accornding to all data**

In [ ]:
boxplot(df_train$subject)
boxplot(df_train$sequence)

In [ ]:
# merger train and train_label data
total <- merge(df_train,df_train_lab,by="sequence")

In [ ]:
library(ggplot2)
library(hrbrthemes)
library(dplyr)
library(tidyr)
library(viridis)
p2 <- ggplot(data=total, aes(x=sequence, group=state)) + geom_density(adjust=1.5, alpha=.4)+theme_ipsum()
p2


**From the result we can see how sequence data are distributed with respect to state data.**

In [ ]:
p2 <- ggplot(data=total, aes(x=subject, group=state)) + geom_density(adjust=1.5, alpha=.4)+ theme_ipsum()
p2

# Plotting sensor data
Train dataset includes data from 13 sensors based on which a participant can be in either of two states (0 or 1).

In [ ]:
# plot for sensor 00
ps00 <- ggplot(data=total, aes(x=sensor_00)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 01
ps01 <- ggplot(data=total, aes(x=sensor_01)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 02
ps02 <- ggplot(data=total, aes(x=sensor_02)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-5, 5)

# plot for sensor 03
ps03 <- ggplot(data=total, aes(x=sensor_03)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-5, 5)

# plot for sensor 04
ps04 <- ggplot(data=total, aes(x=sensor_04)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 05
ps05 <- ggplot(data=total, aes(x=sensor_05)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 06
ps06 <- ggplot(data=total, aes(x=sensor_06)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot sensor 07
ps07 <- ggplot(data=total, aes(x=sensor_07)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 08
ps08 <- ggplot(data=total, aes(x=sensor_08)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 09
ps09 <- ggplot(data=total, aes(x=sensor_09)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 10
ps10 <- ggplot(data=total, aes(x=sensor_10)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 11
ps11 <- ggplot(data=total, aes(x=sensor_11)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-10, 10)

# plot for sensor 12
ps12 <- ggplot(data=total, aes(x=sensor_12)) +
geom_histogram(aes(y=..density..), bins=50, fill='deepskyblue4', col='white') + 
geom_density() +
xlim(-5, 5)

In [ ]:
# setting up plot size
# loading libraries 
library(tidyverse) 
library(gridExtra)
library(TSstudio)
# library(rpart)
library(caTools)
library(ROCR)
options(repr.plot.width=20, repr.plot.height=20)
options(warn=-1)
grid.arrange(ps00, ps01, ps02, ps03,ps04, ps05, ps06, ps07,ps08, ps09, ps10, ps11, ps12, ncol = 3)
options(warn=0)

***We can see from above result that sensor_02 has different characteristics.***

# influence of a sequence on sensor readings

In [ ]:
sen00_seq0 <- total %>%
filter(sequence == 0) %>%
select(sensor_02)

sen00_seq1 <- total %>%
filter(sequence == 1) %>%
select(sensor_02)

sen00_seq1000 <- total %>%
filter(sequence == 1000) %>%
select(sensor_02)

sen00 <- ts(data.frame(sen00_seq0, sen00_seq1, sen00_seq1000))
colnames(sen00) <- c('seq_0', 'seq_1', 'seq_1000')

In [ ]:
ts_plot(sen00, title="Sensor 02 comparison of sequence 0, 1 and 1000", Xtitle="Time")

In [ ]:
# converting numeric values into factor
df_train$sequence <- as.factor(df_train$sequence)
df_train$subject <- as.factor(df_train$subject)
df_train_lab$sequence <- as.factor(df_train_lab$sequence)
df_test$sequence <- as.factor(df_test$sequence)
df_test$subject <- as.factor(df_test$subject)

In [ ]:
# sensor features 
# to plot the correlation graph between sensors
sensors <- paste0('sensor_0', 0:9)
sensors <- c(sensors, paste0('sensor_', 10:12))
sensors

<a id='eda'></a>
# EDA

In [ ]:
# basic stats
summary(df_train)

In [ ]:
# basic stats - test set
summary(df_test)

In [ ]:
# correlation - training data
corrplot::corrplot(cor(df_train[,sensors]))

In [ ]:
# correlation - test set
corrplot::corrplot(cor(df_test[,sensors]))

#### Correlation matrix/plot shows similar pattern as for training data.

In [ ]:
# check mean value of each sensor in training dataset
plot(0:12, colMeans(df_train[sensors]),xlab='sensor', ylab='mean', pch=16, main=paste0('Sensor mean values - Training Data')); grid()
# check mean value of each sensor in testing dataset
plot(0:12, colMeans(df_test[sensors]),xlab='sensor', ylab='mean', pch=16, main=paste0('Sensor mean values - Testing Data')); grid()

<a id='example'></a>
# Look at an Example Sequence

In [ ]:
#Sequence number 3 is taken from the training dataset as an example for further processing
example_sequence <- '3'
df_example <- df_train[df_train$sequence==example_sequence,]
df_example

In [ ]:
# plot time series for each sensor
for (i in sensors) {
  plot(df_example$step, df_example[,i], type='b',
       xlab='step', ylab='',
       main=paste0('Sequence = ',example_sequence,' - ',i))
  grid()
}

In [ ]:
# look at mean value of each sensor
plot(0:12, colMeans(df_example[sensors]),xlab='sensor', ylab='mean', pch=16, main=paste0('Sequence = ',example_sequence,' - mean values')); grid()

In [ ]:
# plotting time series of each sensor in one graph

my_palette = rainbow(13)
plot(df_example$step, df_example$sensor_00,
     col=my_palette[1], type='l',
     xlab='step',ylab='',
     ylim=c(-150,150),
     main=paste0('Sequence = ',example_sequence,' - All sensors'))
for (i in 1:12) {
    s <- sensors[i+1]
    points(df_example$step, df_example[,s], type='l', col=my_palette[i+1])
}
grid()

In [ ]:
# plotting time series of each sensor in one graph - making it more visible

my_palette = rainbow(13)
plot(df_example$step, df_example$sensor_00,
     col=my_palette[1], type='l',
     xlab='step',ylab=,
     ylim=c(-10,10),
     main=paste0('Sequence = ',example_sequence,' - All sensors'))
for (i in 1:12) {
    s <- sensors[i+1]
    points(df_example$step, df_example[,s], type='l', col=my_palette[i+1])
}
grid()

In [ ]:
# correlation between the sensors for the example sequence
#COL2=function(diverging = c("RdBu", "BrBG", "PiYG", "PRGn", "PuOr", "RdYlBu"), n = 200)
install.packages("RColorBrewer")
library(RColorBrewer)
mypalette<-brewer.pal(7,"BrBG")
corrplot::corrplot(cor(df_example[,sensors]), col=mypalette)

In [ ]:
# pairwise scatter plot
options(repr.plot.width = 14, repr.plot.height = 12)
pairs(df_example[,sensors], col='#0000A040')
options(repr.plot.width = 12, repr.plot.height = 6)

<a id='aggregation'></a>
# Aggregation of Data

### Counting subjects

In [ ]:
# subject counts
subject_count_train <- dplyr::group_by(df_train, subject) %>% summarise(n=n())
subject_count_train$n <- subject_count_train$n/60

subject_count_test <- dplyr::group_by(df_test, subject) %>% summarise(n=n())
subject_count_test$n <- subject_count_test$n/60

In [ ]:
head(subject_count_train)

In [ ]:
head(subject_count_test)

### Connect sequence and subject count

In [ ]:
# map sequence => subject
df_subject_map_train <- unique(df_train[,c('sequence','subject')])
# add counts
df_subject_map_train <- dplyr::left_join(df_subject_map_train, subject_count_train, by='subject')
head(df_subject_map_train)

In [ ]:
hist(df_subject_map_train$n)

In [ ]:
# same for test set:

# map sequence => subject
df_subject_map_test <- unique(df_test[,c('sequence','subject')])
# add counts
df_subject_map_test <- dplyr::left_join(df_subject_map_test, subject_count_test, by='subject')
head(df_subject_map_test)

In [ ]:
hist(df_subject_map_test$n)

The main thing turned out to be very important - correlation between state and count of sequences that the subject had. As you can see from my above plot, subjects, who had more than ~95 sequences, were more likely to get target "1" and subjects, who had less than ~25 sequences, were more likely to get target "0". Therefore, dividing the sequences into three groups turned out to be a great idea. 

### Aggregations

In [ ]:
# calc stats
df_train_agg <- dplyr::group_by(df_train, sequence) %>% summarise(
  # mean values
  m_00=mean(sensor_00),
  m_01=mean(sensor_01),
  m_02=mean(sensor_02),
  m_03=mean(sensor_03),
  m_04=mean(sensor_04),
  m_05=mean(sensor_05),
  m_06=mean(sensor_06),
  m_07=mean(sensor_07),
  m_08=mean(sensor_08),
  m_09=mean(sensor_09),
  m_10=mean(sensor_10),
  m_11=mean(sensor_11),
  m_12=mean(sensor_12),
  # standard deviations
  s_00=sd(sensor_00),
  s_01=sd(sensor_01),
  s_02=sd(sensor_02),
  s_03=sd(sensor_03),
  s_04=sd(sensor_04),
  s_05=sd(sensor_05),
  s_06=sd(sensor_06),
  s_07=sd(sensor_07),
  s_08=sd(sensor_08),
  s_09=sd(sensor_09),
  s_10=sd(sensor_10),
  s_11=sd(sensor_11),
  s_12=sd(sensor_12),
  # skewness
  sk_00=moments::skewness(sensor_00),
  sk_01=moments::skewness(sensor_01),
  sk_02=moments::skewness(sensor_02),
  sk_03=moments::skewness(sensor_03),
  sk_04=moments::skewness(sensor_04),
  sk_05=moments::skewness(sensor_05),
  sk_06=moments::skewness(sensor_06),
  sk_07=moments::skewness(sensor_07),
  sk_08=moments::skewness(sensor_08),
  sk_09=moments::skewness(sensor_09),
  sk_10=moments::skewness(sensor_10),
  sk_11=moments::skewness(sensor_11),
  sk_12=moments::skewness(sensor_12),
  # kurtosis
  k_00=moments::kurtosis(sensor_00),
  k_01=moments::kurtosis(sensor_01),
  k_02=moments::kurtosis(sensor_02),
  k_03=moments::kurtosis(sensor_03),
  k_04=moments::kurtosis(sensor_04),
  k_05=moments::kurtosis(sensor_05),
  k_06=moments::kurtosis(sensor_06),
  k_07=moments::kurtosis(sensor_07),
  k_08=moments::kurtosis(sensor_08),
  k_09=moments::kurtosis(sensor_09),
  k_10=moments::kurtosis(sensor_10),
  k_11=moments::kurtosis(sensor_11),
  k_12=moments::kurtosis(sensor_12)
  )

df_train_agg <- as.data.frame(df_train_agg)

In [ ]:
# add subject count info
df_train_agg <- dplyr::left_join(df_train_agg, df_subject_map_train, by='sequence')

# add label
df_train_agg <- dplyr::left_join(df_train_agg, df_train_lab, by='sequence')
# convert to factor
df_train_agg$state <- as.factor(df_train_agg$state)

In [ ]:
head(df_train_agg)

In [ ]:
# define predictors
predictors <- paste0('m_0', 0:9)
predictors <- c(predictors, paste0('m_', 10:12))
predictors <- c(predictors, paste0('s_0', 0:9))
predictors <- c(predictors, paste0('s_', 10:12))
predictors <- c(predictors, paste0('sk_0', 0:9))
predictors <- c(predictors, paste0('sk_', 10:12))
predictors <- c(predictors, paste0('k_0', 0:9))
predictors <- c(predictors, paste0('k_', 10:12))
predictors <- c(predictors, 'n')
print(predictors)

n_pred <- length(predictors)
cat('\nNumber of Predictors:', n_pred)

In [ ]:
# basis stats of aggregated features
summary(df_train_agg[predictors])

#### We get a few NAs in the skewness and kurtosis features. This happens in the cases where the standard deviation in zero.

In [ ]:
# same feature engineering for test set
df_test_agg <- dplyr::group_by(df_test, sequence) %>% summarise(
  # mean values
  m_00=mean(sensor_00),
  m_01=mean(sensor_01),
  m_02=mean(sensor_02),
  m_03=mean(sensor_03),
  m_04=mean(sensor_04),
  m_05=mean(sensor_05),
  m_06=mean(sensor_06),
  m_07=mean(sensor_07),
  m_08=mean(sensor_08),
  m_09=mean(sensor_09),
  m_10=mean(sensor_10),
  m_11=mean(sensor_11),
  m_12=mean(sensor_12),
  # standard deviations
  s_00=sd(sensor_00),
  s_01=sd(sensor_01),
  s_02=sd(sensor_02),
  s_03=sd(sensor_03),
  s_04=sd(sensor_04),
  s_05=sd(sensor_05),
  s_06=sd(sensor_06),
  s_07=sd(sensor_07),
  s_08=sd(sensor_08),
  s_09=sd(sensor_09),
  s_10=sd(sensor_10),
  s_11=sd(sensor_11),
  s_12=sd(sensor_12),
  # skewness
  sk_00=moments::skewness(sensor_00),
  sk_01=moments::skewness(sensor_01),
  sk_02=moments::skewness(sensor_02),
  sk_03=moments::skewness(sensor_03),
  sk_04=moments::skewness(sensor_04),
  sk_05=moments::skewness(sensor_05),
  sk_06=moments::skewness(sensor_06),
  sk_07=moments::skewness(sensor_07),
  sk_08=moments::skewness(sensor_08),
  sk_09=moments::skewness(sensor_09),
  sk_10=moments::skewness(sensor_10),
  sk_11=moments::skewness(sensor_11),
  sk_12=moments::skewness(sensor_12),
  # kurtosis
  k_00=moments::kurtosis(sensor_00),
  k_01=moments::kurtosis(sensor_01),
  k_02=moments::kurtosis(sensor_02),
  k_03=moments::kurtosis(sensor_03),
  k_04=moments::kurtosis(sensor_04),
  k_05=moments::kurtosis(sensor_05),
  k_06=moments::kurtosis(sensor_06),
  k_07=moments::kurtosis(sensor_07),
  k_08=moments::kurtosis(sensor_08),
  k_09=moments::kurtosis(sensor_09),
  k_10=moments::kurtosis(sensor_10),
  k_11=moments::kurtosis(sensor_11),
  k_12=moments::kurtosis(sensor_12)
)

df_test_agg <- as.data.frame(df_test_agg)

In [ ]:
# add subject count info
df_test_agg <- dplyr::left_join(df_test_agg, df_subject_map_test, by='sequence')

In [ ]:
head(df_test_agg)

In [ ]:
# basis stats of aggregated features - for test set
summary(df_test_agg[predictors])

In [ ]:
# feature correlation - training (ignoring NAs in skewness and kurtosis)
options(repr.plot.width = 14, repr.plot.height = 12)
corrplot(cor(df_train_agg[predictors],use = 'complete.obs'))
options(repr.plot.width = 12, repr.plot.height = 6)

In [ ]:
# feature correlation - test set (ignoring NAs in skewness and kurtosis)
options(repr.plot.width = 14, repr.plot.height = 12)
corrplot(cor(df_test_agg[predictors],use = 'complete.obs'))
options(repr.plot.width = 12, repr.plot.height = 6)

### Target:

In [ ]:
# plot target
plot(df_train_agg$state, main='Target (state)'); grid()

#### The target is nicely balanced!

<a id='target_feature'></a>
# Target vs Features

In [ ]:
# define target
target <- 'state'

In [ ]:
# plot target vs features
options(repr.plot.width = 16, repr.plot.height = 5)
for (f in predictors) {
  qqs <- unique(quantile(df_train_agg[,f], seq(0,1,0.1), na.rm=TRUE))
  plot(cut(df_train_agg[,f],qqs), df_train_agg[,target],
       main=paste0('Target vs ',f))
}
options(repr.plot.width = 12, repr.plot.height = 6)

<a id='model'></a>
# Fit Model

In [ ]:
# start H2O
h2o.init()

In [ ]:
# upload data to H2O environment
train_hex <- as.h2o(df_train_agg)
test_hex <- as.h2o(df_test_agg)

In [ ]:
# fit GBM model
n_cv <- 5
set.seed(1234)
t1 <- Sys.time()
fit_GBM <- h2o.gbm(x=predictors, y=target,
                   training_frame=train_hex,
                   nfolds = n_cv,
                   ntrees = 250,
                   learn_rate = 0.05,
                   sample_rate = 1,
                   max_depth = 9,
                   min_rows = 5,
                   col_sample_rate = 0.5,                   
                   stopping_metric = 'AUC',
                   score_each_iteration = TRUE,
                   stopping_rounds = 5,
                   stopping_tolerance = 0.0001,
                   seed=999
)
t2 <- Sys.time()
print(t2-t1)

In [ ]:
# show results of cross validations
fit_GBM@model$cross_validation_metrics_summary

In [ ]:
# plot scoring histories
for (i in 1:n_cv) {
    # get name of i-th CV model
    cv_model_i <- fit_GBM@model$cross_validation_models[[i]]$name
    # access model via name
    fit_temp <- h2o.getModel(cv_model_i)
    # extract scoring history
    score_hist <- fit_temp@model$scoring_history
    # plot history for training / CV
    plot(score_hist$number_of_trees, score_hist$training_auc, 
         col='blue', ylim=c(0.5,1))
    points(score_hist$number_of_trees, score_hist$validation_auc,
           col='orange')
    grid()
}

In [ ]:
# AUC on training data
h2o.auc(fit_GBM, train = TRUE)

In [ ]:
# AUC on cross validations
h2o.auc(fit_GBM, xval = TRUE)

<a id='model_trans'></a>
# Model Transparency

### Feature Importance:

In [ ]:
# variable importance
options(repr.plot.width = 12, repr.plot.height = 12)
h2o.varimp_plot(fit_GBM,num_of_features = 100)
options(repr.plot.width = 12, repr.plot.height = 6)

In [ ]:
# alternative variable importance using SHAP => see direction as well as severity of feature impact
options(repr.plot.width = 16, repr.plot.height = 14)
h2o.shap_summary_plot(model = fit_GBM, newdata=train_hex,
                      top_n_features = n_pred)
options(repr.plot.width = 12, repr.plot.height = 6)

### Partial dependency plots for most important features:

In [ ]:
# get top 10 features
my_features <- h2o.varimp(fit_GBM)$variable[1:10]
my_features

In [ ]:
for (f in my_features) {
    suppressWarnings(
        h2o.partialPlot(fit_GBM, data=train_hex, nbins=50, cols = f)
    )    
}

<a id='submit'></a>
# Apply model on test set and prepare submission

In [ ]:
# calc predictions
pred_test <- as.data.frame(predict(fit_GBM, test_hex))
pred_test <- pred_test$p1
summary(pred_test)

In [ ]:
# show predictions
hist(pred_test,100)

In [ ]:
# prepare and save submission frame
df_sub$state <- pred_test
write_delim(df_sub, file='submission_GBM.csv', delim=',')

In [ ]:
write.csv(df_sub , file = 'Result_JIMZ.csv' , row.names = FALSE )

In [ ]:
# stop H2O
h2o.shutdown(F)